Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Sentiment analysis using BERT and Azure Machine Learning

In this notebook, we fine-tune a pretrained [BERT](https://arxiv.org/abs/1810.04805) model to perform sentiment analysis on the [IMDb Large movie reviews dataset](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) and complete the end-to-end steps to get started with the Azure Machine Learning Python SDK running in Jupyter notebooks. To do this, we:
* Create a machine learning workspace
* Create a cloud-based Jupyter notebook VM with Azure Machine Learning Python SDK installed and pre-configured
* Connect your workspace and create an experiment
* Load, preprocess, and tokenize the data
* Fine-tune BERT for sentiment analysis
* Score and evaluate results

## BERT basics

BERT is a language representation model that effectively captures deep and subtle textual relationships in a corpus. In the [original paper](https://arxiv.org/abs/1810.04805), the authors demonstrate that the BERT model could be easily adapted to build state-of-the-art models for a number of NLP tasks, including text classification, named entity recognition and question answering. 

We fine-tune BERT for sentiment classification by wrapping a pre-trained BERT model with a [sequence classifier](../../utils_nlp/bert/sequence_classification.py). Although we use [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of [BERT](https://github.com/google-research/bert) in this notebook, the [AzureML introduction to BERT](https://github.com/microsoft/AzureML-BERT/blob/master/docs/bert-intro.md) provides a useful overview of BERT concepts.

## IMDb dataset

The [IMDb Large movie reviews](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) dataset for binary sentiment classification contains positive and negative movie reviews. 

## Setup
- Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)

- Go through the [Configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`.

- Install the Python SDK:  make sure to install notebook, and contrib
```
conda create -n azureml -y Python=3.6
source activate azureml
pip install --upgrade azureml-sdk[notebooks,contrib] 
conda install ipywidgets
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```

You will need to restart Jupyter after this.Detailed instructions are [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python) 

Run the cell below to make sure you have installed all necessary packages and to download the dataset.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.dnn import PyTorch
import os
import sys

sys.path.append("../../")

from utils_nlp.dataset.bert_sentiment import download_or_find, extract_data

## Create a workspace

A workspace is a foundational resource in the cloud that you use to experiment, train, and deploy machine learning models. It ties your Azure subscription and resource group to an easily consumed object in the SDK. Initialize your [workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) with a workspace name, Azure region, subscription id, and resource group name.

In [3]:
ws = azureml_utils.get_or_create_workspace(config_path=".")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

BERT_Onboard
eastus
BERT_Onboard


## Create an experiment

Now create an experiment in your workspace. An experiment is another foundational cloud resource that represents a collection of trials (individual model runs). In this tutorial you use the experiment to create runs and track your model training in the Azure portal. Parameters include your workspace reference and a string name for the experiment.

In [4]:
# Creating the experiment (or load the existing one)
EXP_NAME = "bert-sentiment-analysis"

exp = Experiment(ws, name=EXP_NAME)
exp

Name,Workspace,Report Page,Docs Page
bert-sentiment-analysis,BERT_Onboard,Link to Azure Portal,Link to Documentation


## Create or attach an existing compute target

With Azure Machine Learning service, you can train your model on a variety of resources or environments, collectively referred to as [compute targets](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets). In this tutorial, need to create a GPU compute target to fine-tune BERT. 

First, list the virtual machines available in the current workspace.

In [38]:
AmlCompute.supported_vmsizes(ws)

[{'name': 'Standard_DS1_v2', 'vCPUs': 1, 'memoryGB': 3.5},
 {'name': 'Standard_DS2_v2', 'vCPUs': 2, 'memoryGB': 7.0},
 {'name': 'Standard_DS3_v2', 'vCPUs': 4, 'memoryGB': 14.0},
 {'name': 'Standard_DS4_v2', 'vCPUs': 8, 'memoryGB': 28.0},
 {'name': 'Standard_DS5_v2', 'vCPUs': 16, 'memoryGB': 56.0},
 {'name': 'Standard_DS11_v2', 'vCPUs': 2, 'memoryGB': 14.0},
 {'name': 'Standard_DS12_v2', 'vCPUs': 4, 'memoryGB': 28.0},
 {'name': 'Standard_DS13_v2', 'vCPUs': 8, 'memoryGB': 56.0},
 {'name': 'Standard_DS14_v2', 'vCPUs': 16, 'memoryGB': 112.0},
 {'name': 'Standard_DS15_v2', 'vCPUs': 20, 'memoryGB': 140.0},
 {'name': 'Standard_D1_v2', 'vCPUs': 1, 'memoryGB': 3.5},
 {'name': 'Standard_D2_v2', 'vCPUs': 2, 'memoryGB': 7.0},
 {'name': 'Standard_D3_v2', 'vCPUs': 4, 'memoryGB': 14.0},
 {'name': 'Standard_D4_v2', 'vCPUs': 8, 'memoryGB': 28.0},
 {'name': 'Standard_D11_v2', 'vCPUs': 2, 'memoryGB': 14.0},
 {'name': 'Standard_D12_v2', 'vCPUs': 4, 'memoryGB': 28.0},
 {'name': 'Standard_D13_v2', 'vCPUs': 

Next, create an AmlCompute cluster as our training compute resource. If you already have an AmlCompute cluster, this code will use that cluster instead of creating a new one.

In [5]:
COMPUTE_NAME = "gpucluster-bert"
COMPUTE_MIN_NODES = 0
COMPUTE_MAX_NODES = 6

# Try to use the standard CPU VM, but it should be run on GPU instead
VM_SIZE = "Standard_NC6"

if COMPUTE_NAME in ws.compute_targets:
    compute_target = ws.compute_targets[COMPUTE_NAME]
    if compute_target and type(compute_target) == AmlCompute:
        print("====> Found an existing compute target. Using it and not create a new one",
              "ComputeTarget name: " + COMPUTE_NAME, 
              sep="\n")
else:
    print("====> Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                                min_nodes=COMPUTE_MIN_NODES,
                                                                max_nodes=COMPUTE_MAX_NODES)
    
    # Create the cluster or type AmlCompute, based on the provisioning config
    compute_target = ComputeTarget.create(ws, COMPUTE_NAME, provisioning_config)
    
    # Create the cluster
    compute_target.wait_for_completion(show_output=True, 
                                       min_node_count=None,
                                       timeout_in_minutes=20)
    
    # Print out the details
    # Serialize the status to convert the object into JSON format
    print(compute_target.get_status().serialize())
    

====> Found an existing compute target. Using it and not create a new one
ComputeTarget name: gpucluster-bert


## Download the data

To make data accessible for remote training, Azure Machine Learning provides a convenient way to do so via a [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage and interact with it from your remote compute targets. First, download the data locally.

In [6]:
# Download the data to local machine
URL_dataset = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

DATA_FOLDER = "data-cloud"

file_path = download_or_find(URL_dataset, directory=DATA_FOLDER)

=====> Begin downloading
=====> Done downloading


## Upload the data to cloud

Next, upload the data to the datastore.

In [7]:
# Get the default datastore and upload the data
ds = ws.get_default_datastore()

# Checking the connection
print(ds.datastore_type, ds.account_name, ds.container_name, sep="\n")

# Upload the data to cloud
ds.upload(src_dir=DATA_FOLDER, target_path="bert-sentiment-data",
          overwrite=True, show_progress=True)

AzureBlob
bertonboard2588997204
azureml-blobstore-8adaac88-2159-4418-b2c0-a3152749fdc1
Uploading an estimated of 1 files
Uploading data-cloud\aclImdb.tar.gz
Uploaded data-cloud\aclImdb.tar.gz, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_1d9f83f6203c410eb984c42f2bf34f3b

## Make a directory to store the script for running on AmlCompute

Create a directory that will contain all the necessary code from your local machine that you will need to access on the remote resource. This will include the training script and any additional files the training script depends on.

In [8]:
script_folder = os.path.join(os.getcwd(), "bert-sentiment")
os.makedirs(script_folder, exist_ok=True)

## Create the training script

With this training script, you'll:
1. Set up constants
2. Get user parameters
3. Extract the dataset and load it to a dataframe
4. Encode, tokenize, and preprocess training and test data
5. Create a sequence classifier that loads a pre-trained BERT model given the language and number of labels
6. Attach a linear classifier layer to the pre-trained BERT transformer to perform sequence classification
7. Score the test set

See the [local BERT sentiment classification notebook](http://localhost:8888/notebooks/scenarios/text_classification/BERT_sentiment_classification.ipynb) for more detail on these steps.

In [9]:
%%writefile bert-sentiment/train.py

print("======> Script loading =========")

import argparse
import os
import numpy as np
import torch
import torch.nn as nn

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from utils_nlp.dataset.bert_sentiment import extract_data, load_dataset, download_or_find
from utils_nlp.eval.classification import eval_classification
from utils_nlp.models.bert.sequence_classification import BERTSequenceClassifier
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.common.timer import Timer

# Setting up constants
BERT_CACHE_DIR = "./temp"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150
NUM_GPUS = None
TRAIN_SIZE = 0.6
LABEL_COL = "polarity"
TEXT_COL = "sentence"


URL_dataset = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


# Get the user parameters
parser = argparse.ArgumentParser()
parser.add_argument("--data-folder", type=str, dest="data_folder", help="Data folder mounting point")
parser.add_argument("--num-epochs", type=int, dest="num_epochs", default=2, help="Number of epochs")
parser.add_argument("--batch-size", type=int, dest="batch_size", default=32, help="Batch size")

args= parser.parse_args()

BATCH_SIZE = args.batch_size
NUM_EPOCHS = args.num_epochs
DATA_FOLDER = args.data_folder

print("======> Begin download or find file")

file_path = download_or_find(URL_dataset, directory=DATA_FOLDER)


# Extract the dataset
# If you are running the notebook for a second time and already extracted the data, 
# comment out these two lines to save time
dataset_path = extract_data(file_path, DATA_FOLDER)
print("=====> Finish extracting dataset")

# Data folder
dataset_path = os.path.join(DATA_FOLDER, "aclImdb")

# Loading the dataset to dataframe
df_train = load_dataset(os.path.join(dataset_path, "train"))
df_test = load_dataset(os.path.join(dataset_path, "test"))



print("====> Begin encoding label =====")
# Encode the label to be 0 and 1
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(labels_train))
print("=====  Label encoded =====")
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))



tokenizer = Tokenizer(LANGUAGE, to_lower=TO_LOWER, cache_dir=BERT_CACHE_DIR)

print("=====> Tokenizing train data ===========")
tokens_train = tokenizer.tokenize(list(df_train[TEXT_COL]))
print("=====> Tokenizing test data ===========")
tokens_test = tokenizer.tokenize(list(df_test[TEXT_COL]))


# Data preprocessing
print("=====> Preprocessing token trains ========")
tokens_train, mask_train, _ = tokenizer.preprocess_classification_tokens(
    tokens_train, MAX_LEN
)
print("=====> Preprocessing token test ========")
tokens_test, mask_test, _ = tokenizer.preprocess_classification_tokens(
    tokens_test, MAX_LEN
)

print("=====> Create model =============")
# Create model and train
classifier = BERTSequenceClassifier(
    language=LANGUAGE, num_labels=num_labels, cache_dir=BERT_CACHE_DIR
)

print("========> Begin fitting the model=======")
with Timer() as t:
    classifier.fit(
        token_ids=tokens_train,
        input_mask=mask_train,
        labels=labels_train,    
        num_gpus=0,        
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,    
        verbose=True,
    )    
print("[Training time: {:.3f} hrs]".format(t.interval / 3600))


print("=========== Predicting ================")
preds = classifier.predict(
    token_ids=tokens_test, input_mask=mask_test, num_gpus=NUM_GPUS, batch_size=BATCH_SIZE
)

print("============ Prediction result =====================")
print(classification_report(labels_test, preds, target_names=["negative", "positive"]))

Overwriting bert-sentiment/train.py


## Copy utils folder

In [23]:
import shutil
shutil.copytree("../../utils_nlp", os.path.join(script_folder, "utils_nlp"))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\ducl\\Documents\\GitHub\\nlp-2\\scenarios\\text_classification\\bert-sentiment\\utils_nlp'

## Create config file and submit the job to experiment

Create a [PyTorch estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py) to run the fine-tuning script on your compute target. 

In [10]:
# Create a pytorch estimator for the experiment

script_params = {
    "--data-folder": ds.path("bert-sentiment-data").as_mount(),
}

est = PyTorch(source_directory=script_folder,
              compute_target=compute_target,
              entry_script="train.py",
              script_params=script_params,
              use_gpu=True,
              conda_dependencies_file="../../../nlp_gpu.yaml"
             )

WARNING - If environment_definition or conda_dependencies_file is specified, Azure ML will not install any framework related packages on behalf of the user.
WARNING - framework_version is not specified, defaulting to version 1.1.


In [11]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
bert-sentiment-analysis,bert-sentiment-analysis_1565275835_be679511,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


## Monitor the run

View run details in the cells below or by going to the Azure Portal. Following the Link to Azure Portal takes you to the main experiment page. Here you see all the individual runs in the experiment. Any custom-logged values (alpha_value and rmse, in this case) become fields for each run, and also become available for the charts and tiles at the top of the experiment page. To add a logged metric to a chart or tile, hover over it, click the edit button, and find your custom-logged metric.

In [12]:
from azureml.widgets import RunDetails
from azureml.core import Run

In [13]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [14]:
run.wait_for_completion(show_output=True)

RunId: bert-sentiment-analysis_1565275835_be679511
Web View: https://mlworkspace.azure.ai/portal/subscriptions/edc6af13-4b84-402d-bf56-5ed515af6671/resourceGroups/BERT_Onboard/providers/Microsoft.MachineLearningServices/workspaces/BERT_Onboard/experiments/bert-sentiment-analysis/runs/bert-sentiment-analysis_1565275835_be679511

Streaming azureml-logs/20_image_build_log.txt

2019/08/08 14:50:42 Downloading source code...
2019/08/08 14:50:44 Finished downloading source code
2019/08/08 14:50:44 Using acb_vol_4ca06e9b-53c4-4b2d-a33e-984733f2c92d as the home volume
2019/08/08 14:50:44 Creating Docker network: acb_default_network, driver: 'bridge'
2019/08/08 14:50:45 Successfully set up Docker network: acb_default_network
2019/08/08 14:50:45 Setting up Docker configuration...
2019/08/08 14:50:45 Successfully set up Docker configuration
2019/08/08 14:50:45 Logging in to registry: bertonboard8bffe134.azurecr.io
2019/08/08 14:50:46 Successfully logged into bertonboard8bffe134.azurecr.io
2019/08

xz-5.2.4             | 366 KB    |            |   0% 
xz-5.2.4             | 366 KB    | #########7 |  98% 
xz-5.2.4             | 366 KB    | ########## | 100% 

python-3.6.8         | 34.4 MB   |            |   0% 
python-3.6.8         | 34.4 MB   | #6         |  16% 
python-3.6.8         | 34.4 MB   | ###7       |  37% 
python-3.6.8         | 34.4 MB   | #####8     |  59% 
python-3.6.8         | 34.4 MB   | #######5   |  75% 
python-3.6.8         | 34.4 MB   | ########8  |  88% 
python-3.6.8         | 34.4 MB   | #########7 |  98% 
python-3.6.8         | 34.4 MB   | ########## | 100% 

prometheus_client-0. | 42 KB     |            |   0% 
prometheus_client-0. | 42 KB     | ########## | 100% 

llvmlite-0.29.0      | 17.7 MB   |            |   0% 
llvmlite-0.29.0      | 17.7 MB   | ##9        |  29% 
llvmlite-0.29.0      | 17.7 MB   | #######5   |  75% 
llvmlite-0.29.0      | 17.7 MB   | #########4 |  95% 
llvmlite-0.29.0      | 17.7 MB   | ########## | 100% 

jsonschema-3.0.1     | 8

pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  91% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8 MB  | #########1 |  92% 
pytorch-1.1.0        | 376.8

mkl-2019.4           | 204.1 MB  | ########9  |  89% 
mkl-2019.4           | 204.1 MB  | ########9  |  89% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | ########9  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  90% 
mkl-2019.4           | 204.1 MB  | #########  |  91% 
mkl-2019.4           | 204.1 MB  | #########  |  91% 
mkl-2019.4           | 204.1 MB  | #########  |  91% 
mkl-2019.4           | 204.1

cudnn-7.6.0          | 183.3 MB  | #########6 |  97% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  97% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  97% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  97% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  97% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########7 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  98% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  99% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  99% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  99% 
cudnn-7.6.0          | 183.3 MB  | #########8 |  99% 
cudnn-7.6.0          | 183.3

...

 57%|█████▋    | 14240/25000 [04:39<03:33, 50.42it/s]


 76%|███████▋  | 19104/25000 [06:16<01:56, 50.42it/s]


 96%|█████████▌| 23968/25000 [07:52<00:20, 50.18it/s]


100%|█████████▉| 24992/25000 [08:12<00:00, 50.35it/s]
25024it [08:13, 64.16it/s]                           
============ Prediction result =====================
              precision    recall  f1-score   support

    negative       0.85      0.92      0.89     12500
    positive       0.92      0.84      0.88     12500

   micro avg       0.88      0.88      0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service.
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.00028228759765625 seconds

Execution Summary
RunId: bert-sentiment-analysis_1565275835_be679511
Web View: https://mlworkspace.azure.ai/portal/subscriptions/edc6af13-4b84-402d-bf56-5ed515af6671/resourceGroups/BERT_Onboard/providers/Microsoft.MachineLearningServices/workspaces/BERT_Onboard/exp

{'runId': 'bert-sentiment-analysis_1565275835_be679511',
 'target': 'gpucluster-bert',
 'status': 'Completed',
 'startTimeUtc': '2019-08-08T15:23:55.753413Z',
 'endTimeUtc': '2019-08-08T19:52:24.040144Z',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': '7f40114e-08f7-4ba9-944c-bebc67b3ae1c',
  'azureml.git.repository_uri': 'https://github.com/dletk/nlp-2.git',
  'mlflow.source.git.repoURL': 'https://github.com/dletk/nlp-2.git',
  'azureml.git.branch': 'bertAML',
  'mlflow.source.git.branch': 'bertAML',
  'azureml.git.commit': '5e98e8f5040422c6aceb910e46a035995419e5f1',
  'mlflow.source.git.commit': '5e98e8f5040422c6aceb910e46a035995419e5f1',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_abccafd45ae123dff50d256462fb508b'},
 'runDefinition': {'script': 'train.py',
  'arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_db49a3e695b04097b868e947647cbd4d'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'com

In [41]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
bert-sentiment-analysis,bert-sentiment-analysis_1565208521_ecbfdbdf,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation
